Fisher’s exact test - diffential methylation

In [ ]:
import pandas as pd
from scipy.stats import fisher_exact
from statsmodels.stats.multitest import multipletests

# Load merged table with counts
df = pd.read_csv("merged_counts.tsv", sep="\t")

pvals = []
for _, row in df.iterrows():
    table = [
        [row["mod_cov_WT"], row["total_cov_WT"] - row["mod_cov_WT"]],
        [row["mod_cov_MUT"], row["total_cov_MUT"] - row["mod_cov_MUT"]]
    ]
    _, p = fisher_exact(table)
    pvals.append(p)

df["pval"] = pvals
df["pval_adj"] = multipletests(df["pval"], method="fdr_bh")[1]
df["frac_diff"] = (df["mod_cov_MUT"]/df["total_cov_MUT"]) - (df["mod_cov_WT"]/df["total_cov_WT"])

# Filter significant differences
sig = df[(df["pval_adj"] < 0.05) & (df["frac_diff"].abs() >= 0.2)]
sig.to_csv("diff_methylation_sites.tsv", sep="\t", index=False)